# SITCOM-307
This notebook is to support the boresighting of Elanas SHFWS bench on instrument port 1.
It assumes we are starting at the beginning of the night.
We will choose a target in the east and rising to minimize field rotation

#### Import required libraries

In [1]:
import asyncio
import time
import os
import numpy as np
import logging

from lsst.ts import salobj

from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS
from lsst.ts.observatory.control.utils import RotType
from lsst.ts.idl import enums

from lsst.ts.observing.utilities.decorated_logger import DecoratedLogger

In [2]:
logger = DecoratedLogger.get_decorated_logger()
logger.level = logging.DEBUG

#### Instantiate control classes

In [3]:
domain = salobj.Domain()
atcs = ATCS(domain)
latiss = LATISS(domain)
await asyncio.gather(atcs.start_task, latiss.start_task)

[[None, None, None, None, None, None, None], [None, None, None, None]]

Verify that ATAOS corrections are active and dome is following

In [ ]:
await atcs.rem.ataos.cmd_enableCorrection.set_start(m1=True, hexapod=True, atspectrograph=True)
await atcs.enable_dome_following()

## Using LATISS, perform a CWFS then a boresight alignment
this can be done using the scriptQueue
The CWFS and boresight alignment require a ~8th magnitude star, but the SHWFS will use a very bright star (Agena (Hadar), 0.55 mag, technically a double according to simbad?)

In [ ]:
cwfs_target = "HIP 67909"
shwfs_target = "Agena" #  aka Hadar

### Perform CWFS with LATISS

In [ ]:
# ADD PART FOR DOING CWFS - this will be scripts sent to the scriptQueue

### Perform Boresighting with LATISS via scriptQueue

acq_exposure_time: 2.0
acq_filter: SDSSr
acq_grating: empty_1
do_acquire: true
do_pointing_model: true
max_acq_iter: 3
object_name: cwfs_target
target_pointing_tolerance: 5.0
target_pointing_verification: true

In [ ]:
# ADD PART FOR BORESIGHT ALIGNMENT -- this will be scripts sent to the scriptQueue

## Switch Instrument Ports, Will now be using the SHWFS and instrument

In [ ]:
atcs.stop_tracking()

In [ ]:
# Can't use aget or next due to a callback function being used somewhere...
# tmp = await atcs.rem.atmcs.tel_mount_AzEl_Encoders.next(flush=True, timeout=5)
tmp = atcs.rem.atmcs.tel_mount_AzEl_Encoders.get()

In [ ]:
curr_az=tmp.azimuthCalculatedAngle[0]
curr_el=tmp.elevationCalculatedAngle[0]

#### Move telescope to change instrument ports with M3

In [ ]:
atcs.point_azel(az=curr_az, el=70)

#### Change M3 to instrument port 1

In [ ]:
await atcs.rem.atmcs.cmd_setInstrumentPort.set_start(port=enums.ATMCS.M3ExitPort.NASMYTH1)

### Slew to the SHWFS target
Because Elana will need to manually adjust the acquisition camera in the next steps, we'll fix the rotator for now. <br>
This means that the offset axes determination will have to be done fairly quickly. <br>
After this slew, there are three offsets that need to be accounted for and corrected.
1. The offset between the boresight for LATISS and for INST1.
2. The pointing offset between LATISS and INST1
3. The acquisition camera will have an unknown physical offset. <br>
Also, the camera will be out of focus.

In [ ]:
await atcs.slew_object(cwfs_target, rot_type=RotType.Physical, rot=0)

### Elana physically moves the acquisition camera to have the star at the center
At this point, the camera will be at the offset position of 1+2

### Elana manually focuses the acquisition camera, ensuring it stays at the center
acquisition camera is 11.3 mm by 7.1 mm <br>
Pixel size is 5.86 um <br>
Telescope focal length is 21.6m <br>
field is 1.9 by 1.2 arcminutes <br>
pixel scale is 0.001 arcsec/pixel <br>

### Perform X/Y offsets to map the detector axes to sky coordinates

At this point, it is imperative that Elana write down how the star moves on the detector.<br>
Strongly suggest using a sticky note and putting arrows for X and Y. <br>
If possible it would be useful to rotate the acquisition camera so there is not a rotation to account for <br>

In [ ]:
# get the current offsets
starting_offsets = await atcs.rem.atptg.evt_offsetSummary.aget(timeout=5)
print(starting_offsets)

In [ ]:
# Start with positive X-offset of 1/4 of the short side of the detector
offset = np.round(1.2*60*0.25)
print(f'offset will be {offset} arcseconds')

In [ ]:
atcs.offset_xy(x=offset,y=0)

In [ ]:
#now come back
atcs.offset_xy(x=-offset,y=0)

In [ ]:
#now move in Y
atcs.offset_xy(x=0,y=offset)

In [ ]:
#now come back to the starting position
atcs.offset_xy(x=0,y=-offset)

In [ ]:
# Check the offsets are consistent with the originals
current_offsets = await atcs.rem.atptg.evt_offsetSummary.aget(timeout=5)
print(current_offsets)

## Now take an image with the rotator moving to make an arc
The center of the arc will be the boresight of the telescope. <br>
The detector will need to be physically moved such that the center of the arc is on the center of the detector <br> 

In [ ]:
### Slew to the target again, but this time have the instrument start rotating

In [ ]:
await atcs.slew_object(cwfs_target, rot_type=RotType.PhysicalSky, rot=0)

### Now offset the telescope to put the star at the center of the detector
Write down the final offsets!!

In [ ]:
atcs.offset_xy(x=0,y=0)

In [ ]:
final_offsets = await atcs.rem.atptg.evt_offsetSummary.aget(timeout=5)
print(final_offsets)

#### Absorb the offsets such that the telescope is boresight is now aligned with the camera

In [ ]:
await atcs.rem.atptg.cmd_offsetAbsorb.set_start(num=0)
await atcs.rem.atptg.cmd_offsetAbsorb.set_start(num=1)
await atcs.rem.atptg.cmd_poriginAbsorb.set_start(num=0)
await atcs.rem.atptg.cmd_poriginAbsorb.set_start(num=1)

In [ ]:
# Not sure these are useful, but they should be zero.
post_absorbed_offsets = await atcs.rem.atptg.evt_offsetSummary.aget(timeout=5)
print(post_absorbed_offsets)